In [14]:
import numpy as np
import pandas as pd

In [16]:
import pandas as pd
import numpy as np

# --- 1. Load and Prepare Data ---
# Set the minimum number of pass attempts required to be included in the analysis
MIN_ATTEMPTS = 3

# Attempt to load the dataset for Week 1
try:
    df = pd.read_csv('dataset/train/input_2023_w01.csv')
except FileNotFoundError:
    print("The file 'dataset/train/input_2023_w01.csv' was not found.")
    print("Please ensure the file path is correct.")
    # Exit if the data isn't found
    exit()

# --- 2. Isolate Relevant Plays and Frames ---
pass_plays = df[df['player_role'] == 'Passer'][['game_id', 'play_id']].drop_duplicates()
df_filtered = pd.merge(df, pass_plays, on=['game_id', 'play_id'])

max_frames = df_filtered.groupby(['game_id', 'play_id'])['frame_id'].transform('max')
last_frame_df = df_filtered[df_filtered['frame_id'] == max_frames]

# --- 3. Calculate Score and Log Targets for Each Play ---
play_results = []

for (game_id, play_id), play_df in last_frame_df.groupby(['game_id', 'play_id']):
    if 'Passer' not in play_df['player_role'].values or 'Defense' not in play_df['player_side'].values:
        continue

    qb_name = play_df[play_df['player_role'] == 'Passer']['player_name'].iloc[0]

    # Find the targeted receiver for this play
    target_df = play_df[play_df['player_role'] == 'Targeted Receiver']
    # Ensure a target exists before proceeding
    if not target_df.empty:
        targeted_receiver = target_df['player_name'].iloc[0]
    else:
        # If no targeted receiver is listed, skip the play
        continue

    # Calculate distance to nearest defender
    ball_land_x = play_df['ball_land_x'].iloc[0]
    ball_land_y = play_df['ball_land_y'].iloc[0]
    ball_coords = np.array([ball_land_x, ball_land_y])
    
    defenders_df = play_df[play_df['player_side'] == 'Defense']
    defender_coords = defenders_df[['x', 'y']].values
    
    distances = np.linalg.norm(defender_coords - ball_coords, axis=1)
    min_distance = np.min(distances)

    play_results.append({
        'qb_name': qb_name,
        'score': min_distance,
        'targeted_receiver': targeted_receiver
    })

# --- 4. Aggregate and Display Results ---
if play_results:
    results_df = pd.DataFrame(play_results)

    # --- NEW: Apply Minimum Attempts Threshold ---
    pass_counts = results_df['qb_name'].value_counts()
    valid_qbs = pass_counts[pass_counts >= MIN_ATTEMPTS].index.tolist()
    
    # Filter the results to only include valid QBs
    filtered_results = results_df[results_df['qb_name'].isin(valid_qbs)].copy()

    # --- NEW: Find Most Targeted Receiver for Each QB ---
    target_counts = filtered_results.groupby(['qb_name', 'targeted_receiver']).size().reset_index(name='count')
    # For each qb, find the row with the maximum target count
    most_targeted = target_counts.loc[target_counts.groupby('qb_name')['count'].idxmax()]

    # Calculate the average score for the filtered QBs
    avg_scores = filtered_results.groupby('qb_name')['score'].mean().reset_index()
    
    # Merge the average scores with the most targeted receiver info
    final_df = pd.merge(avg_scores, most_targeted[['qb_name', 'targeted_receiver']], on='qb_name')

    # Rename columns for clarity
    final_df.columns = ['Quarterback', 'Average Score', 'Most Targeted Receiver']
    
    # Sort the final results
    sorted_final_df = final_df.sort_values(by='Average Score', ascending=False).reset_index(drop=True)

    print(f"Average Completion Score per Quarterback (Min {MIN_ATTEMPTS} Attempts) - Week 1")
    print(sorted_final_df)
else:
    print("No valid pass plays were found to score.")

Average Completion Score per Quarterback (Min 3 Attempts) - Week 1
           Quarterback  Average Score Most Targeted Receiver
0          Jalen Hurts      10.132705             A.J. Brown
1           Derek Carr       9.718100            Chris Olave
2          Jordan Love       9.542379            Jayden Reed
3        Justin Fields       9.384822              Cole Kmet
4           Joe Burrow       9.210657          Ja'Marr Chase
5       Deshaun Watson       9.111093           Amari Cooper
6            Mac Jones       8.798553        Kendrick Bourne
7       Justin Herbert       8.644235           Keenan Allen
8        Lamar Jackson       8.396475            Zay Flowers
9      Patrick Mahomes       8.362855              Noah Gray
10    Matthew Stafford       8.111192             Puka Nacua
11      Baker Mayfield       7.989583             Mike Evans
12      Tua Tagovailoa       7.942793            Tyreek Hill
13      Desmond Ridder       7.795275             Kyle Pitts
14        Kirk Cou

In [17]:
sorted_final_df

,Quarterback,Average Score,Most Targeted Receiver
0,Jalen Hurts,10.132705,A.J. Brown
1,Derek Carr,9.718100,Chris Olave
2,Jordan Love,9.542379,Jayden Reed
3,Justin Fields,9.384822,Cole Kmet
4,Joe Burrow,9.210657,Ja'Marr Chase
5,Deshaun Watson,9.111093,Amari Cooper
6,Mac Jones,8.798553,Kendrick Bourne
7,Justin Herbert,8.644235,Keenan Allen
8,Lamar Jackson,8.396475,Zay Flowers
9,Patrick Mahomes,8.362855,Noah Gray
